In [98]:
require 'daru'
require 'daru_plotting_plotly'
require 'hg'
require 'rbplotly'

class Numeric
  def mili
    (self * 10 ** (-3)).to_f
  end
  
  def micro
    (self * 10 ** (-6)).to_f
  end
end

Daru.plotting_library = :plotly

:plotly

In [143]:
Hg::Hg.create do |gen|
  gen.headers :orm, :v_mean, :v_diff
  gen.body <<BODY
470, 4.15, 30.mili
1000, 8.8, 60.mili
1500, 12.95, 1250.mili
2200, 13.7, 3
3300, 13.9, 2900.mili
1330, 11.7, 120.mili
1830, 13.6, 3100.mili
1600, 13.2, 2200.mili
1720, 13.4, 3000.mili
1430, 12.4, 840.mili
BODY
end.daru_dataframe.tap do |df|
  df.sort! [:orm]
  df.add_vector :ripple, df.map_rows { |row| row[:v_diff].to_f / row[:v_mean].to_f }
  df.add_vector :i_mean, df.map_rows { |row| row[:v_mean].to_f / row[:orm].to_f }
  df.plot(x: :orm, y: :i_mean, layout: { title: '出力電流' }).show
  df.plot(x: :orm, y: :ripple, layout: { title: 'リップル含有率' }).show
end.write_csv("data/jikken_4/data.csv")

#<Plotly::Offline::HTML:0x007fbbd9339d60 @id="2c052d7d-e20d-4bdd-9be0-62addcf6c087", @data=[{:x=>[470, 1000, 1330, 1430, 1500, 1600, 1720, 1830, 2200, 3300], :y=>[0.008829787234042553, 0.0088, 0.008796992481203006, 0.008671328671328672, 0.008633333333333333, 0.00825, 0.007790697674418605, 0.007431693989071038, 0.006227272727272727, 0.004212121212121212], :type=>:scatter, :mode=>"", :name=>:i_mean}], @layout={:width=>1000, :height=>500, :title=>"出力電流"}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fbbda0f8550 @id="2ce7eede-a19c-40a8-bcb8-3ac1b9191150", @data=[{:x=>[470, 1000, 1330, 1430, 1500, 1600, 1720, 1830, 2200, 3300], :y=>[0.007228915662650602, 0.006818181818181817, 0.010256410256410256, 0.06774193548387096, 0.09652509652509653, 0.16666666666666669, 0.22388059701492538, 0.22794117647058826, 0.21897810218978103, 0.20863309352517984], :type=>:scatter, :mode=>"", :name=>:ripple}], @layout={:width=>1000, :height=>500, :title=>"リップル含有率"}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

In [135]:
Daru::DataFrame.from_csv('data/jikken_1.csv')

#<Daru::DataFrame(11x4)>
                     i     v_diff     v_mean     ripple
          0      0.227       19.2       11.8 1.62711864
          1     0.2052       19.1      11.85 1.61181434
          2     0.1825       19.5       12.0      1.625
          3     0.1699       19.5       12.0      1.625
          4     0.1468       19.6      12.05 1.62655601
          5     0.1228       19.8       12.1 1.63636363
          6     0.0989       19.8       12.2 1.62295081
          7     0.0742       19.9      12.21 1.62981162
          8     0.0502       19.9      12.25 1.62448979
          9     0.0258         20       12.5        1.6
         10          0         20       12.6 1.58730158

In [126]:
trace = ['47', '470', '1000', '2200'].map do |orm|
  df = Daru::DataFrame.from_csv("data/jikken_3/#{orm}.csv")
  {x: df['ei'].to_a, y: df['i'].to_a, name: "#{orm}Ω"}
end
Plotly::Plot.new(data: trace, layout: { xaxis: { title: 'Ei(V)' }, yaxis: { title: 'Io(A)' } }).show

#<Plotly::Offline::HTML:0x007fbbd899e580 @id="77d9f28b-98b5-4c38-ae20-cdfe358548d1", @data=[{:x=>[3, 4, 5, 5.5, 6, 7, 8, 9, 11, 12], :y=>[0.004893617021276596, 0.006808510638297872, 0.007659574468085106, 0.008085106382978723, 0.008297872340425533, 0.00851063829787234, 0.008723404255319148, 0.008723404255319148, 0.008936170212765958, 0.008936170212765958], :name=>"47Ω"}, {:x=>[5, 6, 7, 8, 8.5, 9, 9.5, 10, 12, 14, 15], :y=>[0.004404255319148936, 0.005319148936170213, 0.006276595744680851, 0.007872340425531916, 0.008404255319148936, 0.00848936170212766, 0.00851063829787234, 0.00853191489361702, 0.008659574468085107, 0.008723404255319148, 0.00874468085106383], :name=>"470Ω"}, {:x=>[7, 9, 10, 11, 12, 12.5, 13, 15], :y=>[0.00365, 0.005019999999999999, 0.005849999999999999, 0.006730000000000001, 0.00775, 0.008119999999999999, 0.008199999999999999, 0.0083], :name=>"1000Ω"}, {:x=>[6, 8, 10, 12, 13, 14, 15, 16], :y=>[0.001681818181818182, 0.0022727272727272726, 0.0028636363636363633, 0.0036363636363636364, 0.004045454545454546, 0.004454545454545455, 0.004931818181818181, 0.005272727272727273], :name=>"2200Ω"}], @layout={:xaxis=>{:title=>"Ei(V)"}, :yaxis=>{:title=>"Io(A)"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>